In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df.sample(3)

,question,answer
73,Which country is famous for its kangaroos?,Australia
53,Which metal is a liquid at room temperature?,Mercury
55,Who discovered electricity?,Benjamin-Franklin


In [7]:
# first we have to tokenize the data
def tokenize(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'","")
  tokens = text.split()
  return tokens

In [10]:
# form vocab
vocab = {"<UNK>":0}
def build_vocab(row):
  t_question = tokenize(row["question"])
  t_answers = tokenize(row["answer"])
  # print(t_question,t_answers)
  t_merged = t_question + t_answers
  for word in t_merged:
    if word not in vocab:
      vocab[word] = len(vocab)



df.apply(build_vocab,axis=1)
print(vocab)

{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'capital': 4, 'of': 5, 'france': 6, 'paris': 7, 'germany': 8, 'berlin': 9, 'who': 10, 'wrote': 11, 'to': 12, 'kill': 13, 'a': 14, 'mockingbird': 15, 'harper-lee': 16, 'largest': 17, 'planet': 18, 'in': 19, 'our': 20, 'solar': 21, 'system': 22, 'jupiter': 23, 'boiling': 24, 'point': 25, 'water': 26, 'celsius': 27, '100': 28, 'painted': 29, 'mona': 30, 'lisa': 31, 'leonardo-da-vinci': 32, 'square': 33, 'root': 34, '64': 35, '8': 36, 'chemical': 37, 'symbol': 38, 'for': 39, 'gold': 40, 'au': 41, 'which': 42, 'year': 43, 'did': 44, 'world': 45, 'war': 46, 'ii': 47, 'end': 48, '1945': 49, 'longest': 50, 'river': 51, 'nile': 52, 'japan': 53, 'tokyo': 54, 'developed': 55, 'theory': 56, 'relativity': 57, 'albert-einstein': 58, 'freezing': 59, 'fahrenheit': 60, '32': 61, 'known': 62, 'as': 63, 'red': 64, 'mars': 65, 'author': 66, '1984': 67, 'george-orwell': 68, 'currency': 69, 'united': 70, 'kingdom': 71, 'pound': 72, 'india': 73, 'delhi': 74, 'discov

In [12]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

text_to_indices("What i am", vocab)


[1, 0, 0]

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

dataset = QADataset(df, vocab)

In [18]:
dataset[2]

(tensor([10, 11, 12, 13, 14, 15]), tensor([16]))

In [23]:
dataloader = DataLoader(dataset,batch_size=1,pin_memory=True,num_workers=4,persistent_workers=True,shuffle = True)
for question, answer in dataloader:
  print(question,answer)


tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([[74]])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([[72]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([[170]])
tensor([[ 10,  75, 111]]) tensor([[112]])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([[100]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([[185]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]]) tensor([[121]])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([[7]])
tensor([[ 10,  11, 157, 158, 159]]) tensor([[160]])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([[298]])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([[91]])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([[321]])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([[287]])
tensor([[ 10,  11, 189, 158, 190]]) tensor

In [70]:
class MyRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [71]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [77]:
learning_rate = 0.001
epochs = 59

In [78]:
model = MyRNN(len(vocab))

In [79]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [80]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 521.920822
Epoch: 2, Loss: 456.549385
Epoch: 3, Loss: 376.501875
Epoch: 4, Loss: 313.115099
Epoch: 5, Loss: 259.632473
Epoch: 6, Loss: 210.664635
Epoch: 7, Loss: 167.667641
Epoch: 8, Loss: 129.319575
Epoch: 9, Loss: 98.458597
Epoch: 10, Loss: 75.364047
Epoch: 11, Loss: 58.360164
Epoch: 12, Loss: 45.668046
Epoch: 13, Loss: 36.405738
Epoch: 14, Loss: 29.562914
Epoch: 15, Loss: 24.349096
Epoch: 16, Loss: 20.103891
Epoch: 17, Loss: 16.996031
Epoch: 18, Loss: 14.495673
Epoch: 19, Loss: 12.615321
Epoch: 20, Loss: 10.819872
Epoch: 21, Loss: 9.437519
Epoch: 22, Loss: 8.332198
Epoch: 23, Loss: 7.378990
Epoch: 24, Loss: 6.601406
Epoch: 25, Loss: 5.922889
Epoch: 26, Loss: 5.338025
Epoch: 27, Loss: 4.830600
Epoch: 28, Loss: 4.410539
Epoch: 29, Loss: 4.014954
Epoch: 30, Loss: 3.675094
Epoch: 31, Loss: 3.380269
Epoch: 32, Loss: 3.122388
Epoch: 33, Loss: 2.884819
Epoch: 34, Loss: 2.663214
Epoch: 35, Loss: 2.475001
Epoch: 36, Loss: 2.298948
Epoch: 37, Loss: 2.137778
Epoch: 38, Loss: 1.

In [91]:
def predict(model, question, threshold=0.5):
    # convert question to numbers
    numerical_question = text_to_indices(question, vocab)
    # tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient computation for inference
        # send to model - capture both outputs
        output = model(question_tensor)  # Only take the first output, ignore hidden

        # convert logits to probs
        probs = torch.nn.functional.softmax(output, dim=1)

        # find index of max prob
        value, index = torch.max(probs, dim=1)

        if value < threshold:
            print("I don't know")
        else:
            # Convert tensor to int for indexing
            predicted_word = list(vocab.keys())[index.item()]
            print(predicted_word)

In [92]:
predict(model, "What is?",0.5)

I don't know
